In [17]:
# pip install folium

In [24]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# 1. 데이터 로드
# Main dataset: 서울시 한강공원 월별 이용객 현황
main_data_path = 'seoul_utf8.csv'
main_data = pd.read_csv(main_data_path, encoding='utf-8')

# Coordinates dataset: 서울특별시 한강공원 좌표 정보
coordinates_data_path = '서울특별시_한강공원_한강사진정보_20240831.csv'
coordinates_data = pd.read_csv(coordinates_data_path, encoding='utf-8')

# 2. 좌표 데이터 정제
# Extract park names from keywords in the coordinates dataset
coordinates_data['공원명'] = coordinates_data['사진 키워드'].str.extract(r'(\w+한강공원)')
coordinates_data = coordinates_data.dropna(subset=['공원명'])
coordinates_data = coordinates_data.groupby('공원명', as_index=False).first()
coordinates_data = coordinates_data[['공원명', '사진 위도', '사진 경도']].rename(
    columns={'사진 위도': '위도', '사진 경도': '경도'}
)

# 3. 데이터 병합
# Merge main data with coordinates on 공원명
merged_data = pd.merge(main_data, coordinates_data, on='공원명', how='inner')

# 4. 시각화 데이터 준비
visualization_data = merged_data[['공원명', '위도', '경도', '공원 구명']]

# Add average visitor data for CircleMarker (if available)
if '일반이용자(아침)' in merged_data.columns and '일반이용자(낮)' in merged_data.columns and '일반이용자(저녁)' in merged_data.columns:
    visualization_data['평균 이용객'] = (merged_data['일반이용자(아침)'] +
                                     merged_data['일반이용자(낮)'] +
                                     merged_data['일반이용자(저녁)']) / 3
else:
    # Add mock data if average visitor data is missing
    visualization_data['평균 이용객'] = [10000, 15000, 20000, 5000, 12000, 18000, 25000, 8000, 11000, 9000, 16000]



# 5. MarkerCluster 지도 생성
map_cluster = folium.Map(location=[37.5665, 126.9780], zoom_start=11)

# 클러스터 크기에 따른 색상 설정
cluster_colors = {
    'small': 'orange',  # 100개 미만
    'medium': '#FFC0CB',  # 100개 이상, 200개 미만
    'large': 'red'      # 200개 이상
}

# JavaScript 함수 정의
icon_create_function = f"""
function(cluster) {{
    var childCount = cluster.getChildCount();
    var c = ' marker-cluster-';
    var color = '';
    
    // 클러스터 크기에 따라 색상 설정
    if (childCount < 100) {{
        c += 'small';
        color = '{cluster_colors['small']}';
    }} else if (childCount < 200) {{
        c += 'medium';
        color = '{cluster_colors['medium']}';
    }} else {{
        c += 'large';
        color = '{cluster_colors['large']}';
    }}
    
    return new L.DivIcon({{
        html: '<div style="background-color:' + color + ';"><span>' + childCount + '</span></div>',
        className: 'marker-cluster' + c,
        iconSize: new L.Point(40, 40)
    }});
}}
"""


# MarkerCluster를 생성하고 줌 레벨 조건 설정
marker_cluster = MarkerCluster(
    icon_create_function=icon_create_function,
    options={
        'disableClusteringAtZoom': 14  # 줌 레벨 15 이상에서는 클러스터링 비활성화
    }
).add_to(map_cluster)

# 마커 추가
from folium import Icon

for _, row in visualization_data.iterrows():
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=f"{row['공원명']} - 자치구: {row['공원 구명']}",
        icon=Icon(color="blue", icon="info-sign")  # 핀 스타일 설정
    ).add_to(marker_cluster)

# 지도 저장
map_cluster.save('map_cluster.html')



# 6. CircleMarker 지도 생성
map_circle = folium.Map(location=[37.5665, 126.9780], zoom_start=11)

for _, row in visualization_data.iterrows():
    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=row['평균 이용객'] / 10000,
        color='blue',
        fill=False,
        weight=2,
        fill_opacity=0.6,
        popup=f"{row['공원명']} - 평균 이용객: {row['평균 이용객']}"
    ).add_to(map_circle)

map_circle.save('map_circle.html')

# 7. FeatureGroup 지도 생성
map_group = folium.Map(location=[37.5665, 126.9780], zoom_start=11)

for district in visualization_data['공원 구명'].unique():
    feature_group = folium.FeatureGroup(name=district)
    district_parks = visualization_data[visualization_data['공원 구명'] == district]
    
    for _, row in district_parks.iterrows():
        folium.Marker(
            location=[row['위도'], row['경도']],
            popup=f"{row['공원명']} - 자치구: {district}"
        ).add_to(feature_group)
    
    map_group.add_child(feature_group)

folium.LayerControl().add_to(map_group)
map_group.save('map_group.html')


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_32200\1486023972.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  visualization_data['평균 이용객'] = (merged_data['일반이용자(아침)'] +


In [25]:
print(visualization_data)


         공원명         위도          경도 공원 구명         평균 이용객
0    광나루한강공원  37.544878  127.112761   강동구   20690.000000
1     이촌한강공원  37.530460  126.950537   용산구   30330.666667
2     뚝섬한강공원  37.529677  127.067157   광진구  122656.666667
3     잠실한강공원  37.527402  127.099510   송파구   33390.000000
4     양화한강공원  37.539475  126.900519  영등포구   25713.333333
..       ...        ...         ...   ...            ...
810   잠실한강공원  37.527402  127.099510   송파구   37220.000000
811  여의도한강공원  37.530092  126.933635  영등포구   77206.666667
812   양화한강공원  37.539475  126.900519  영등포구   20014.666667
813   뚝섬한강공원  37.529677  127.067157   광진구  207900.000000
814   강서한강공원  37.586767  126.816877   강서구   16244.333333

[815 rows x 5 columns]


In [26]:
print(visualization_data[['위도', '경도']])


            위도          경도
0    37.544878  127.112761
1    37.530460  126.950537
2    37.529677  127.067157
3    37.527402  127.099510
4    37.539475  126.900519
..         ...         ...
810  37.527402  127.099510
811  37.530092  126.933635
812  37.539475  126.900519
813  37.529677  127.067157
814  37.586767  126.816877

[815 rows x 2 columns]


In [27]:
print(merged_data)

     현황 일련번호     공원 코드      공원명 공원 시명 공원 구명    공원 지번 주소      공원 도로명 주소  \
0      87623  Hzone001  광나루한강공원    서울   강동구   암사동 637-6     선사로 83-106   
1      87629  Hzone006   이촌한강공원    서울   용산구  이촌동 302-17      이촌로72길 62   
2      87626  Hzone003   뚝섬한강공원    서울   광진구   자양동 427-1      강변북로 2273   
3      87625  Hzone002   잠실한강공원    서울   송파구     잠실동 1-1        한가람로 65   
4      87631  Hzone008   양화한강공원    서울  영등포구    당산동 98-1        노들로 221   
..       ...       ...      ...   ...   ...         ...            ...   
810    86798  Hzone002   잠실한강공원    서울   송파구     잠실동 1-1        한가람로 65   
811    86802  Hzone007  여의도한강공원    서울  영등포구   여의도동 85-1       여의동로 280   
812    86801  Hzone008   양화한강공원    서울  영등포구    당산동 98-1        노들로 221   
813    86800  Hzone003   뚝섬한강공원    서울   광진구   자양동 427-1      강변북로 2273   
814    86794  Hzone011   강서한강공원    서울   강서구      방화동 47  양천로27길 279-23   

     등록자 일련번호                  등록 일시  수정자 일련번호  ... 난지 하늘다리  갈대숲탐장로  꿀벌숲  \
0          83  2024-03-08 10:40:48.

In [28]:
print(visualization_data[['위도', '경도']].isnull().sum())


위도    0
경도    0
dtype: int64


In [29]:
print(visualization_data[['위도', '경도']])


            위도          경도
0    37.544878  127.112761
1    37.530460  126.950537
2    37.529677  127.067157
3    37.527402  127.099510
4    37.539475  126.900519
..         ...         ...
810  37.527402  127.099510
811  37.530092  126.933635
812  37.539475  126.900519
813  37.529677  127.067157
814  37.586767  126.816877

[815 rows x 2 columns]


In [30]:
print(visualization_data[['공원명', '평균 이용객']])


         공원명         평균 이용객
0    광나루한강공원   20690.000000
1     이촌한강공원   30330.666667
2     뚝섬한강공원  122656.666667
3     잠실한강공원   33390.000000
4     양화한강공원   25713.333333
..       ...            ...
810   잠실한강공원   37220.000000
811  여의도한강공원   77206.666667
812   양화한강공원   20014.666667
813   뚝섬한강공원  207900.000000
814   강서한강공원   16244.333333

[815 rows x 2 columns]


In [31]:
main_data

,현황 일련번호,공원 코드,공원명,공원 시명,공원 구명,공원 지번 주소,공원 도로명 주소,등록자 일련번호,등록 일시,수정자 일련번호,...,강변물놀이장,강변프롬나드,난지 하늘다리,갈대숲탐장로,꿀벌숲,치유의숲,그라스정원,노들섬,습지생태공원,현황 일시
0,87623,Hzone001,광나루한강공원,서울,강동구,암사동 637-6,선사로 83-106,83,2024-03-08 10:40:48.0,NaN,...,0,0,0,0,0,0,0,0,0,2024-02-29 00:00:00.0
1,87629,Hzone006,이촌한강공원,서울,용산구,이촌동 302-17,이촌로72길 62,83,2024-03-08 14:14:33.0,NaN,...,0,0,0,0,0,0,0,19198,0,2024-02-29 00:00:00.0
2,87626,Hzone003,뚝섬한강공원,서울,광진구,자양동 427-1,강변북로 2273,83,2024-03-08 11:29:41.0,83.0,...,0,0,0,0,0,0,0,0,0,2024-02-29 00:00:00.0
3,87625,Hzone002,잠실한강공원,서울,송파구,잠실동 1-1,한가람로 65,83,2024-03-08 10:44:40.0,NaN,...,0,0,0,0,0,0,0,0,0,2024-02-29 00:00:00.0
4,87631,Hzone008,양화한강공원,서울,영등포구,당산동 98-1,노들로 221,83,2024-03-08 14:40:51.0,NaN,...,0,0,0,0,0,0,0,0,0,2024-02-29 00:00:00.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
810,86798,Hzone002,잠실한강공원,서울,송파구,잠실동 1-1,한가람로 65,5,2018-01-31 00:00:00.0,NaN,...,0,0,0,0,0,0,0,0,0,2018-01-31 00:00:00.0
811,86802,Hzone007,여의도한강공원,서울,영등포구,여의도동 85-1,여의동로 280,10,2018-01-31 00:00:00.0,NaN,...,0,0,0,0,0,0,0,0,0,2018-01-31 00:00:00.0
812,86801,Hzone008,양화한강공원,서울,영등포구,당산동 98-1,노들로 221,11,2018-01-31 00:00:00.0,NaN,...,0,0,0,0,0,0,0,0,0,2018-01-31 00:00:00.0
813,86800,Hzone003,뚝섬한강공원,서울,광진구,자양동 427-1,강변북로 2273,6,2018-01-31 00:00:00.0,NaN,...,0,0,0,0,0,0,0,0,0,2018-01-31 00:00:00.0


In [32]:
print(visualization_data.head())

       공원명         위도          경도 공원 구명         평균 이용객
0  광나루한강공원  37.544878  127.112761   강동구   20690.000000
1   이촌한강공원  37.530460  126.950537   용산구   30330.666667
2   뚝섬한강공원  37.529677  127.067157   광진구  122656.666667
3   잠실한강공원  37.527402  127.099510   송파구   33390.000000
4   양화한강공원  37.539475  126.900519  영등포구   25713.333333


In [33]:
main_data

,현황 일련번호,공원 코드,공원명,공원 시명,공원 구명,공원 지번 주소,공원 도로명 주소,등록자 일련번호,등록 일시,수정자 일련번호,...,강변물놀이장,강변프롬나드,난지 하늘다리,갈대숲탐장로,꿀벌숲,치유의숲,그라스정원,노들섬,습지생태공원,현황 일시
0,87623,Hzone001,광나루한강공원,서울,강동구,암사동 637-6,선사로 83-106,83,2024-03-08 10:40:48.0,NaN,...,0,0,0,0,0,0,0,0,0,2024-02-29 00:00:00.0
1,87629,Hzone006,이촌한강공원,서울,용산구,이촌동 302-17,이촌로72길 62,83,2024-03-08 14:14:33.0,NaN,...,0,0,0,0,0,0,0,19198,0,2024-02-29 00:00:00.0
2,87626,Hzone003,뚝섬한강공원,서울,광진구,자양동 427-1,강변북로 2273,83,2024-03-08 11:29:41.0,83.0,...,0,0,0,0,0,0,0,0,0,2024-02-29 00:00:00.0
3,87625,Hzone002,잠실한강공원,서울,송파구,잠실동 1-1,한가람로 65,83,2024-03-08 10:44:40.0,NaN,...,0,0,0,0,0,0,0,0,0,2024-02-29 00:00:00.0
4,87631,Hzone008,양화한강공원,서울,영등포구,당산동 98-1,노들로 221,83,2024-03-08 14:40:51.0,NaN,...,0,0,0,0,0,0,0,0,0,2024-02-29 00:00:00.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
810,86798,Hzone002,잠실한강공원,서울,송파구,잠실동 1-1,한가람로 65,5,2018-01-31 00:00:00.0,NaN,...,0,0,0,0,0,0,0,0,0,2018-01-31 00:00:00.0
811,86802,Hzone007,여의도한강공원,서울,영등포구,여의도동 85-1,여의동로 280,10,2018-01-31 00:00:00.0,NaN,...,0,0,0,0,0,0,0,0,0,2018-01-31 00:00:00.0
812,86801,Hzone008,양화한강공원,서울,영등포구,당산동 98-1,노들로 221,11,2018-01-31 00:00:00.0,NaN,...,0,0,0,0,0,0,0,0,0,2018-01-31 00:00:00.0
813,86800,Hzone003,뚝섬한강공원,서울,광진구,자양동 427-1,강변북로 2273,6,2018-01-31 00:00:00.0,NaN,...,0,0,0,0,0,0,0,0,0,2018-01-31 00:00:00.0


In [34]:
print(main_data.columns)

Index(['현황 일련번호', '공원 코드', '공원명', '공원 시명', '공원 구명', '공원 지번 주소', '공원 도로명 주소',
       '등록자 일련번호', '등록 일시', '수정자 일련번호', '수정 일시', '일반이용자(아침)', '일반이용자(낮)',
       '일반이용자(저녁)', '자전거', '인라인', 'pm(개인형이동장치)', '주요행사', '마라톤', '운동시설', '야구장',
       '론볼링장', '트랙구장', '롤러장', '자전거공원', '외국인', '수상시설', '수영장/물놀이장', '빙상장/눈설매장',
       '전망쉼터', '캠핑장', '자연학습장', '음악분수', '키즈랜드', '장미원', 'x게임장', '자벌레', '달빛무지개',
       '세빛섬', '수상무대', '계절,녹음수광장', '천상계단', '피아노물길', '멀티프라자', '서울색공원', '물빛광장',
       '너플들판테라스', '골프장', '여의도샛강', '여의도시민 요트나루', '평화공원브릿지', '거울분수', '강변물놀이장',
       '강변프롬나드', '난지 하늘다리', '갈대숲탐장로', '꿀벌숲', '치유의숲', '그라스정원', '노들섬', '습지생태공원',
       '현황 일시'],
      dtype='object')


In [35]:
main_data.columns

Index(['현황 일련번호', '공원 코드', '공원명', '공원 시명', '공원 구명', '공원 지번 주소', '공원 도로명 주소',
       '등록자 일련번호', '등록 일시', '수정자 일련번호', '수정 일시', '일반이용자(아침)', '일반이용자(낮)',
       '일반이용자(저녁)', '자전거', '인라인', 'pm(개인형이동장치)', '주요행사', '마라톤', '운동시설', '야구장',
       '론볼링장', '트랙구장', '롤러장', '자전거공원', '외국인', '수상시설', '수영장/물놀이장', '빙상장/눈설매장',
       '전망쉼터', '캠핑장', '자연학습장', '음악분수', '키즈랜드', '장미원', 'x게임장', '자벌레', '달빛무지개',
       '세빛섬', '수상무대', '계절,녹음수광장', '천상계단', '피아노물길', '멀티프라자', '서울색공원', '물빛광장',
       '너플들판테라스', '골프장', '여의도샛강', '여의도시민 요트나루', '평화공원브릿지', '거울분수', '강변물놀이장',
       '강변프롬나드', '난지 하늘다리', '갈대숲탐장로', '꿀벌숲', '치유의숲', '그라스정원', '노들섬', '습지생태공원',
       '현황 일시'],
      dtype='object')